In [69]:
import sys
from pathlib import Path

import pandas as pd

current_dir = Path('.')
current_dir = current_dir.absolute()
root_dir = current_dir.parent.parent
src_dir = root_dir / 'src'

sys.path.append(str(src_dir))

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv(str(src_dir / '.env'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [70]:
from api.containers.teams_and_winner import LoadTeamsAndWinnerOnly

service = LoadTeamsAndWinnerOnly().service()

df = service.collect_all()
df

F:\Projects\hltv_notebook\src\infra\data_frames\pipeline\matches.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[C.Winner] = winners


,Team1,Team2,Winner,Map
0,Budapest Five,Katuna,Katuna,inf Inferno
1,Katuna,Budapest Five,Budapest Five,anc Ancient
2,Budapest Five,Katuna,Katuna,ovp Overpass
3,ECSTATIC,Astralis Talent,Astralis Talent,inf Inferno
4,Eternal Fire,fnatic,fnatic,ovp Overpass
...,...,...,...,...
45,Spirit,Virtus.pro,Spirit,ovp Overpass
46,Spirit,Virtus.pro,Spirit,d2 Dust2
47,Spirit,Virtus.pro,Virtus.pro,inf Inferno
48,FORZE,Gambit,Gambit,d2 Dust2


In [71]:
from infra.data_frames.pipeline import C

team_1_unique = set(df[C.Team1].unique())
team_2_unique = set(df[C.Team2].unique())
id_to_team = dict(enumerate(team_1_unique.union(team_2_unique), start=1))
team_to_id = {v: k for k, v in id_to_team.items()}

maps = set(df[C.Map].unique())
id_to_map = dict(enumerate(maps, start=1))
map_to_id = {v: k for k, v in id_to_map.items()}


df = pd.DataFrame({
    'Team': df[C.Team1].map(team_to_id),
    'Opponent': df[C.Team2].map(team_to_id),
    C.Map: df[C.Map].map(map_to_id),
    'Win': df[C.Winner] == df[C.Team1],

})

df

,Team,Opponent,Map,Win
0,58,181,3,False
1,181,58,10,False
2,58,181,4,False
3,597,1243,3,False
4,1060,1208,4,False
...,...,...,...,...
45,1266,806,4,True
46,1266,806,1,True
47,1266,806,3,False
48,1241,908,1,False


In [72]:
from sklearn.model_selection import train_test_split

X = df[['Team', 'Opponent', C.Map]]
y = df['Win']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=2)

X_train

,Team,Opponent,Map
14,1066,1101,1
16,248,478,7
7,306,184,7
48,749,103,3
18,1239,14,11
...,...,...,...
49,974,614,4
49,622,803,10
37,651,448,1
25,1180,719,3


In [73]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, min_samples_leaf=4)
model.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=4, random_state=0)

In [74]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_train)
accuracy_score(y_train, y_pred)

0.8245475676573136

In [75]:
model.predict([[1, 2, 1]])

F:\Projects\hltv_notebook\venv\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([False])

In [76]:
import numpy as np
from sklearn.model_selection import cross_val_score

accuracy = np.mean(cross_val_score(model, X_test, y_test, scoring='accuracy')) * 100
accuracy

50.23341943917089

In [77]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted Loss', 'Predicted Win'],
    index=['True Loss', 'True Win']
)

ValueError: Found input variables with inconsistent numbers of samples: [1268, 24092]